# Lab 12-3 many to many fixed
### hihello training 
* many to many
* fixed input sequence length

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from pprint import pprint

print(tf.__version__)

1.12.0


### Prepairing dataset
hih -> ihe, ihe -> hel, hel -> ell, ell -> llo

In [2]:
strings = 'hihello'
char_set = sorted(list(set('hihello' + 'hello')))
idx2char = {idx : char for idx, char in enumerate(char_set)}
char2idx = {char : idx for idx, char in enumerate(char_set)}

In [3]:
data = []
for i in range(len(strings) -2):
    data.append(strings[i:i+3])

x_strings = data[0:4]
y_strings = data[1:5]

print(x_strings, y_strings)

['hih', 'ihe', 'hel', 'ell'] ['ihe', 'hel', 'ell', 'llo']


In [4]:
x_data = list(map(lambda strings : [char2idx.get(char) for char in strings], x_strings))
y_data = list(map(lambda strings : [char2idx.get(char) for char in strings], y_strings))

# data 형태확인
print(x_data)
print(y_data)

[[1, 2, 1], [2, 1, 0], [1, 0, 3], [0, 3, 3]]
[[2, 1, 0], [1, 0, 3], [0, 3, 3], [3, 3, 4]]


In [5]:
# input
tr_dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
tr_dataset = tr_dataset.batch(batch_size = 2).repeat()

### Creating model

In [6]:
num_classes = len(char2idx)
hidden_dim = 10
max_sequence = 3

input_dim = len(char2idx)
output_dim = len(char2idx)
one_hot = np.eye(len(char2idx))

model = keras.Sequential()
model.add(keras.layers.Embedding(input_dim=input_dim, output_dim=output_dim,
                                 trainable=False, input_length=max_sequence,
                                 embeddings_initializer=keras.initializers.Constant(one_hot)))
model.add(keras.layers.SimpleRNN(units=hidden_dim, return_sequences=True))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(units=num_classes, activation='softmax')))

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3, 5)              25        
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 3, 10)             160       
_________________________________________________________________
time_distributed (TimeDistri (None, 3, 5)              55        
Total params: 240
Trainable params: 215
Non-trainable params: 25
_________________________________________________________________


### Training model

In [8]:
# Creating a training operation
lr = .01
opt = tf.train.GradientDescentOptimizer(learning_rate = lr)
model.compile(optimizer=opt,
              loss=keras.losses.sparse_categorical_crossentropy)

### Checking performance

In [9]:
model.fit(tr_dataset, epochs=30, steps_per_epoch= 4 // 2)

ValueError: Error when checking target: expected time_distributed to have 3 dimensions, but got array with shape (None, 3)

### Checking performance